In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, LeakyReLU
from keras.losses import mse, binary_crossentropy
from keras.utils import np_utils, plot_model
from keras import backend as K
from keras import optimizers
from keras import regularizers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/train_labels.csv
/kaggle/input/audio-binary-classification/test_data.npy


In [2]:
train_data = np.load('/kaggle/input/audio-binary-classification/train_data.npy');
test_data = np.load('/kaggle/input/audio-binary-classification/test_data.npy');


print("inputs loaded")

inputs loaded


In [3]:
#Scale and Reshape
train_data = train_data.astype('float32');
test_data = test_data.astype('float32');

#Normalize Data
#train_data /= np.amax(test_data);
#test_data /= np.amax(test_data);

print(train_data.shape);

train_data = train_data.reshape(-1,44100,1);
test_data = test_data.reshape(-1,44100,1);

input_shape = train_data.shape;
print(input_shape);

(3999, 44100)
(3999, 44100, 1)


In [4]:
#Look at CSV and store as array
train_labels = pd.read_csv("/kaggle/input/audio-binary-classification/train_labels.csv");
train_labels = train_labels.values;
train_labels = train_labels[:,1];
print(train_labels[:10]);
print(train_labels.shape)

[0 0 0 1 0 0 1 0 1 0]
(3999,)


In [5]:
'''
#Modify data
x_data = np.zeros( (train_data.shape[0] * 2 + 250, train_data.shape[1],1) );
y_data = np.zeros( (train_labels.shape[0] * 2 + 250))

x_data[:3999,:,0] = train_data[:3999,:,0];
y_data[:3999] = train_labels[:3999];

for a in range(train_data.shape[0]):
    s = np.random.normal(1,.02,44100);
    x_data[a+3999,:,0] = np.multiply(x_data[a,:,0],s);
    y_data[a+3999] = train_labels[a];
        
b = 0;

for a in range(train_data.shape[0]):
    if(train_labels[a] == 1):
        s = np.random.normal(1,.02,44100);
        x_data[2*3999+b,:,0] = np.multiply(x_data[a,:,0],s);
        y_data[2*3999+b] = train_labels[a];
        b += 1;
    if(b >= 250):
        break;
        
print("data made noisey")
'''

'\n#Modify data\nx_data = np.zeros( (train_data.shape[0] * 2 + 250, train_data.shape[1],1) );\ny_data = np.zeros( (train_labels.shape[0] * 2 + 250))\n\nx_data[:3999,:,0] = train_data[:3999,:,0];\ny_data[:3999] = train_labels[:3999];\n\nfor a in range(train_data.shape[0]):\n    s = np.random.normal(1,.02,44100);\n    x_data[a+3999,:,0] = np.multiply(x_data[a,:,0],s);\n    y_data[a+3999] = train_labels[a];\n        \nb = 0;\n\nfor a in range(train_data.shape[0]):\n    if(train_labels[a] == 1):\n        s = np.random.normal(1,.02,44100);\n        x_data[2*3999+b,:,0] = np.multiply(x_data[a,:,0],s);\n        y_data[2*3999+b] = train_labels[a];\n        b += 1;\n    if(b >= 250):\n        break;\n        \nprint("data made noisey")\n'

In [6]:
x_data = train_data;
y_data = train_labels;

In [7]:
#Reshape the data and make this bad a 2D bad boy
x_data = x_data.reshape(-1,210,210,1);
test_data = test_data.reshape(-1,210,210,1);

In [8]:
# create convolution neural network
model = Sequential()
model.add(Conv2D(128, kernel_size=(6,6),
                 input_shape=(210,210,1)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Dropout(0.45))

model.add(Conv2D(64, (3,3)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.33))

model.add(Flatten())
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 205, 205, 128)     4736      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 205, 205, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 51, 51, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 51, 51, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 49, 64)        73792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 49, 49, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)        0         
__________

In [9]:
#Compile model
sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer='sgd', 
              metrics=['accuracy'])

In [10]:
history = model.fit(x_data,y_data, shuffle=True,
          batch_size=30,epochs=500,verbose=1)

Epoch 1/500
3999/3999 [==============================] - 9s 2ms/step - loss: 0.1313 - acc: 0.8725
Epoch 2/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.1096 - acc: 0.8772
Epoch 3/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0999 - acc: 0.8780
Epoch 4/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0859 - acc: 0.8882
Epoch 5/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0795 - acc: 0.8980
Epoch 6/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0739 - acc: 0.9070
Epoch 7/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0719 - acc: 0.9097
Epoch 8/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0694 - acc: 0.9140
Epoch 9/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0675 - acc: 0.9205
Epoch 10/500
3999/3999 [==============================] - 6s 2ms/step - loss: 0.0662 - acc: 0.9165
Epoch 11/500
3999/3

In [11]:
Y_pred = model.predict(test_data);
print(Y_pred)
df_pred = pd.DataFrame(Y_pred);
df_pred.index.name = "Id"
df_pred = pd.DataFrame.rename(df_pred,columns={0:"Label"})
print(df_pred);
df_pred.to_csv("output.csv")

[[2.6524067e-06]
 [4.8723817e-04]
 [3.8199127e-03]
 ...
 [3.0279160e-05]
 [0.0000000e+00]
 [9.8915917e-01]]
             Label
Id                
0     2.652407e-06
1     4.872382e-04
2     3.819913e-03
3     6.854534e-07
4     5.960464e-07
...            ...
3992  3.289849e-03
3993  1.192093e-07
3994  3.027916e-05
3995  0.000000e+00
3996  9.891592e-01

[3997 rows x 1 columns]
